# NLP with CNN

### Exercise objectives:

- Use CNN instead of RNN for NLP

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
######################################
### Run this cell to load the data ###
######################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2023-02-24 14:44:28.653979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 14:44:31.613864: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-24 14:44:31.613880: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-24 14:44:36.942045: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-24 14:44:40.163152: W tensorflow/stream_executor/platform/de

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXWL12D/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXWL12D/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2023-02-24 14:45:24.517520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-24 14:45:24.517605: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-24 14:45:24.517643: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (FR-VE-200901-01): /proc/driver/nvidia/version does not exist
2023-02-24 14:45:24.517972: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Remember that to do NLP, you need to go through one of the following options, as shown here: 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. We will go into both, starting from the one on the left were the embedding is learned in the network.




# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). So, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolution on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2 dimensional as the filters can move up and down, and left and right. In the case of our sentences, we want the kernel to move _only_ in the word by word direction (The alternative, moving coordinate by coordinate of the embedding space doesn't make much sense).

So let's create a model that use convolutions.

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post', maxlen=150)
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post', maxlen=150)

## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: `input_dim` is the `vocab_size + 1`, `output_dim` is the embedding space dimension, and `mask_zero` has to be set to `True`. Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words (tokens) each kernel is taking into account ;)

In [5]:
from tensorflow.keras import layers, Sequential

In [14]:
# Size of your embedding space = size of the vector representing each word
embedding_size = 50

model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # 16 +1 for the 0 padding
    input_length=150, # Max_sentence_length (optional, for model summary)
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer :)
))
model.add(layers.Conv1D(20, kernel_size=3))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 150, 50)           1521000   
                                                                 
 conv1d_4 (Conv1D)           (None, 148, 20)           3020      
                                                                 
 flatten_4 (Flatten)         (None, 2960)              0         
                                                                 
 dense_4 (Dense)             (None, 10)                29610     
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,553,641
Trainable params: 1,553,641
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Look at the number of parameters. You can compare it to the model that you had in previous exercise (esp. the first one)

In [15]:
model.compile(loss='binary_crossentropy', # No need to OHE target
              optimizer='rmsprop',
              metrics=['accuracy'])

❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 10ms/step - loss: 0.0093 - accuracy: 0.9994 - val_loss: 0.5391 - val_accuracy: 0.7720
Epoch 2/20
55/55 [==============================] - 0s 9ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.5781 - val_accuracy: 0.7773
Epoch 3/20
55/55 [==============================] - 0s 8ms/step - loss: 1.1393e-04 - accuracy: 1.0000 - val_loss: 0.6616 - val_accuracy: 0.7760
Epoch 4/20
55/55 [==============================] - 0s 9ms/step - loss: 7.2664e-05 - accuracy: 1.0000 - val_loss: 0.7089 - val_accuracy: 0.7840
Epoch 5/20
55/55 [==============================] - 0s 8ms/step - loss: 6.2680e-06 - accuracy: 1.0000 - val_loss: 0.7500 - val_accuracy: 0.7787
Epoch 6/20
55/55 [==============================] - 0s 8ms/step - loss: 1.4806e-06 - accuracy: 1.0000 - val_loss: 0.8004 - val_accuracy: 0.7707
The accuracy evaluated on the test set is of 76.840%


# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D`🔥 

In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure: 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture as on the right side.

❓ **Question** ❓ Learn a word2vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long, it prepares you for real world challenges - but you have already done all the building bricks in the previous exercises

In [18]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [19]:
word2vec_transfer = api.load("glove-wiki-gigaword-50")

In [22]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [24]:
# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=150)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=150)

❓ **Question** ❓ Now construct a model that has a `Conv1D` layer, a flatten layer, a dense layer, and an output layer. Compile it, and fit it on the train data. You can then evaluate it on the test set.

In [26]:
model2 = Sequential()

model2.add(layers.Masking(mask_value=0, input_shape=X_train_pad_2[0].shape))
model2.add(layers.Conv1D(20, kernel_size=3))
model2.add(layers.Flatten())
model2.add(layers.Dense(10, activation="relu"))
model2.add(layers.Dense(1, activation="sigmoid"))
model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 150, 50)           0         
                                                                 
 conv1d_6 (Conv1D)           (None, 148, 20)           3020      
                                                                 
 flatten_6 (Flatten)         (None, 2960)              0         
                                                                 
 dense_8 (Dense)             (None, 10)                29610     
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 32,641
Trainable params: 32,641
Non-trainable params: 0
_________________________________________________________________


In [27]:
model2.compile(loss='binary_crossentropy', # No need to OHE target
              optimizer='rmsprop',
              metrics=['accuracy'])

model2.fit(X_train_pad_2, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


Epoch 1/20
55/55 [==============================] - 11s 198ms/step - loss: 0.7024 - accuracy: 0.4949 - val_loss: 0.6933 - val_accuracy: 0.4733
Epoch 2/20
55/55 [==============================] - 1s 17ms/step - loss: 0.6909 - accuracy: 0.5189 - val_loss: 0.6864 - val_accuracy: 0.5720
Epoch 3/20
55/55 [==============================] - 0s 7ms/step - loss: 0.6528 - accuracy: 0.6400 - val_loss: 0.6376 - val_accuracy: 0.6533
Epoch 4/20
55/55 [==============================] - 0s 7ms/step - loss: 0.5649 - accuracy: 0.7137 - val_loss: 0.6295 - val_accuracy: 0.6547
Epoch 5/20
55/55 [==============================] - 0s 7ms/step - loss: 0.4630 - accuracy: 0.8051 - val_loss: 0.6685 - val_accuracy: 0.6653
Epoch 6/20
55/55 [==============================] - 0s 6ms/step - loss: 0.3665 - accuracy: 0.8451 - val_loss: 0.6310 - val_accuracy: 0.6707
Epoch 7/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2774 - accuracy: 0.9057 - val_loss: 0.7002 - val_accuracy: 0.6627
Epoch 8/20
55/55

In [28]:
model2.evaluate(X_test_pad_2, y_test)

79/79 [==============================] - 0s 4ms/step - loss: 0.6365 - accuracy: 0.6388


[0.6365195512771606, 0.6388000249862671]

❓ **Question** ❓ You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on.

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)